# Word Embeddings Generation

This Jupyter notebook demonstrates how to generate word embeddings from a given corpus using a pre-trained BERT model. These embeddings will be used to find semantically similar matches for a user query.

# Notebook Overview
- Import Dependencies
- Configure Logging and Define Constants and Paths
- Load and Preprocess Data
- Initialize BERT Tokenizer and Model
- Generate Embeddings in Batches
- Save Embeddings to File

# Import Dependencies

In [1]:
import sys
import os  
from pathlib import Path  
import logging
from datetime import datetime

# Data manipulation libraries
import pandas as pd
import numpy as np

# Deep learning framework
import torch  

# NLP libraries
import nltk  # Natural Language Toolkit
from nemo.collections.nlp.models import BERTLMModel  # BERT Language Model from NVIDIA NeMo
from transformers import AutoTokenizer  # Tokenizer for transformer-based models

# Configure Logging and Define Constants and Paths

In [2]:
# Create logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %I:%M:%S %p")  # 12-hour format with AM/PM

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

CORPUS_PATH = "../data/raw/corpus.csv"
BERT_MODEL_NAME = "bert-large-uncased"
BERT_MODEL_DATAFABRIC_PATH = "/home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo"
EMBEDDINGS_OUTPUT_PATH = "../data/processed/embeddings.csv"

In [3]:
logger.info('Notebook execution started.')

2025-04-01 03:34:06 PM - INFO - Notebook execution started.


# Load and Preprocess Data

In [4]:
# Download the Punkt tokenizer data for sentence tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# Load the dataset into a Pandas DataFrame
corpus_df = pd.read_csv(CORPUS_PATH)

# Display the first few rows of the DataFrame
logger.info("First few entries of the DataFrame:")
print(corpus_df.head())

2025-04-01 03:34:07 PM - INFO - First few entries of the DataFrame:


   Unnamed: 0  Topic                                             Pledge
0           0      1  Actually we as an association are still pretty...
1           1      1  EFFAT welcomes the Commission Proposal for a R...
2           2      1  HOTREC calls for a level playing field and fai...
3           3      1  Estonia sees the need to synchronize and harmo...
4           4      1  Sphere Travel Club contributes to a flourishin...


In [6]:
documents = corpus_df["Pledge"].astype(str).tolist()  # Convert the column to a list

# Initialize BERT Tokenizer and Model

In [7]:
# Initialize the tokenizer with a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info("Loading BERT model...")

# Ensure you have added the 'bertlargeuncased' model from the NVIDIA NGC model catalog.
# If unavailable, use the alternative method below to download the model online.

# Uncomment the following line to download the BERT model online:
# bert_model = BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False).to(device)

# Load the BERT model from a local .nemo file inside datafabric folder
bert_model = BERTLMModel.restore_from(BERT_MODEL_DATAFABRIC_PATH, strict=False).to(device)

logger.info("BERT model loaded successfully.")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2025-04-01 03:34:08 PM - INFO - Loading BERT model...
[NeMo W 2025-04-01 15:35:42 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: /home/yzhang/data/nlp/bert/47316/hdf5/lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus/training/
    max_predictions_per_seq: 80
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    


...

[NeMo W 2025-04-01 15:36:04 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2025-04-01 15:36:04 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-04-01 15:36:04 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-04-01 15:36:06 save_restore_connector:249] Model BERTLMModel was successfully restored from /home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo.


2025-04-01 03:36:06 PM - INFO - BERT model loaded successfully.


# Generate Embeddings in Batches

In [8]:
def generate_embeddings_in_batches(texts, tokenizer, model, batch_size=32):
    """
    Generates text embeddings using the NeMo BERT model in batches.
    
    Args:
        texts (list of str): List of input texts.
        tokenizer: Pretrained tokenizer.
        model: Pretrained NeMo BERT model.
        batch_size (int, optional): Batch size for processing. Default is 32.
    
    Returns:
        np.ndarray: Generated embeddings.
    """
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize batch with padding and truncation
        encoded_input = tokenizer(
            batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
        )
        encoded_input = {key: val.to(device) for key, val in encoded_input.items()}

        with torch.no_grad():  # Disable gradient computation for inference
            output = model.bert_model(**encoded_input)
        
        # Extract the CLS token representation for embeddings
        embeddings = output[:, 0, :].cpu().numpy()  # CLS token representation
        all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# Save Embeddings to File

In [9]:
# Generate embeddings using the pre-trained model
embeddings = generate_embeddings_in_batches(documents, tokenizer, bert_model)

# Convert embeddings into a DataFrame
df_embeddings = pd.DataFrame(embeddings)

# Define output file path for saving embeddings
df_embeddings.to_csv(EMBEDDINGS_OUTPUT_PATH, index=False)

logger.info(f"✅ Embedding completed and saved to: {EMBEDDINGS_OUTPUT_PATH}")

2025-04-01 03:36:26 PM - INFO - ✅ Embedding completed and saved to: ../data/processed/embeddings.csv


In [10]:
logger.info('Notebook execution completed.')

2025-04-01 03:36:26 PM - INFO - Notebook execution completed.
